In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from collections import defaultdict
from datetime import date, timedelta
from sklearn import metrics
from utils import get_state_error_df, get_predictions_df

In [2]:
labels = pd.read_csv('election_data/final_results_by_state.csv').rename(columns={'dem_vote_share': 'd_prob'})
battleground_df = pd.read_csv('label_data/2016_pres_labels_battleground.csv')[['state_po', 'battleground']]

In [3]:
d_probs = pd.read_csv('poll_data/all_pres_polls_d_probs.csv')

In [119]:
start_date = '2020-05-01'
end_dates = list(map(lambda day: str((date(2020, 11, 3) - timedelta(day))), [3, 50, 100]))

In [124]:
models = {
    'ewma_03_lw': {'alpha': 0.3, 'logistic': True},
    'ewma_03': {'alpha': 0.3, 'logistic': False},
    'ewma_01_lw': {'alpha': 0.1, 'logistic': True},
    'ewma_01': {'alpha': 0.1, 'logistic': False},
    'ewma_1': {'alpha': 1, 'logistic': False},
}

In [125]:
model_errors = list()
for model in models:
    for end_date in end_dates:
        preds = get_predictions_df(d_probs, **models[model], start_date=start_date, end_date=end_date)
        model_er = pd.merge(get_state_error_df(labels, preds), battleground_df).sort_values(by=['battleground', 'state_po'])
        battleground_err = np.mean(model_er[model_er['battleground']]['error'])
        non_battleground_err = np.mean(model_er[~model_er['battleground']]['error'])
        model_errors.append((model, end_date, True, battleground_err))
        model_errors.append((model, end_date, False, non_battleground_err))
        fig = px.bar(model_er, x='state_po', y='error', color='battleground',
                     labels={'error': 'Error', 'state_po': 'State'}, 
                     title=f'Model {model} Signed Error ({start_date} - {end_date}) (Negative Means Democrat Optimistic)')
        filename = f'{model}_{start_date}-{end_date}_signed_error'
        fig.write_image(f'figures/{filename}.png')
        model_er.to_csv(f'figures/data/{filename}.csv', index=False)
        open(f'figures/tables/{filename}.tex', 'w+').write(model_er.to_latex(index=False, caption=f'Model {model} Signed Error ({start_date} - {end_date})\n(Negative Means Democrat Optimistic)'))
        print(f'Wrote {filename} to ./figures')

Wrote ewma_03_lw_2020-05-01-2020-10-31_signed_error to ./figures
Wrote ewma_03_lw_2020-05-01-2020-09-14_signed_error to ./figures
Wrote ewma_03_lw_2020-05-01-2020-07-26_signed_error to ./figures
Wrote ewma_03_2020-05-01-2020-10-31_signed_error to ./figures
Wrote ewma_03_2020-05-01-2020-09-14_signed_error to ./figures
Wrote ewma_03_2020-05-01-2020-07-26_signed_error to ./figures
Wrote ewma_01_lw_2020-05-01-2020-10-31_signed_error to ./figures
Wrote ewma_01_lw_2020-05-01-2020-09-14_signed_error to ./figures
Wrote ewma_01_lw_2020-05-01-2020-07-26_signed_error to ./figures
Wrote ewma_01_2020-05-01-2020-10-31_signed_error to ./figures
Wrote ewma_01_2020-05-01-2020-09-14_signed_error to ./figures
Wrote ewma_01_2020-05-01-2020-07-26_signed_error to ./figures
Wrote ewma_1_2020-05-01-2020-10-31_signed_error to ./figures
Wrote ewma_1_2020-05-01-2020-09-14_signed_error to ./figures
Wrote ewma_1_2020-05-01-2020-07-26_signed_error to ./figures


In [126]:
df = pd.DataFrame(model_errors, columns=['model', 'end_date', 'battleground', 'error'])

In [127]:
nb_df = df[~df['battleground']]
fig = px.line(nb_df, x="end_date", y="error", color="model",
              labels={'end_date': 'Prediction Date', 'error': 'Signed Error'},
             title='Non-battleground State Signed Error\n(Negative Means Democrat Optimistic)')
# fig.show()
filename = f'signed_error_non_battleground_time_series'
fig.write_image(f'figures/{filename}.png')
open(f'figures/tables/{filename}.tex', 'w+').write(nb_df.to_latex(index=False, caption=f'Non-battleground State Signed Error\n(Negative Means Democrat Optimistic)'))
print(f'Wrote to {filename}')

Wrote to signed_error_non_battleground_time_series


In [128]:
b_df = df[df['battleground']]
fig = px.line(b_df, x="end_date", y="error", color="model",
              labels={'end_date': 'Prediction Date', 'error': 'Signed Error'},
             title='Battleground State Signed Error\n(Negative Means Democrat Optimistic)')
# fig.show()
filename = f'signed_error_battleground_time_series'
fig.write_image(f'figures/{filename}.png')
open(f'figures/tables/{filename}.tex', 'w+').write(b_df.to_latex(index=False, caption=f'Battleground State Signed Error\n(Negative Means Democrat Optimistic)'))
b_df.to_csv(f'figures/data/{filename}.csv', index=False)
print(f'Wrote to {filename}')

Wrote to signed_error_battleground_time_series


In [129]:
for model in models:
    model_df = df[df['model'] == model]
    fig = px.line(model_df, x="end_date", y="error", color="battleground",
                  labels={'end_date': 'Prediction Date', 'error': 'Signed Error'},
                 title=f'{model} Signed Error\n(Negative Means Democrat Optimistic)')
#     fig.show()
    filename = f'{model}_signed_error_time_series'
    fig.write_image('figures/' + filename + '.png')
    model_df.to_csv('figures/data/' + filename + '.csv', index=False)
    open(f'figures/tables/{filename}.tex', 'w+').write(model_df.to_latex(index=False, caption=f'{model} Signed Error\n(Negative Means Democrat Optimistic)'))
    print(f'Wrote to {filename}')

Wrote to ewma_03_lw_signed_error_time_series
Wrote to ewma_03_signed_error_time_series
Wrote to ewma_01_lw_signed_error_time_series
Wrote to ewma_01_signed_error_time_series
Wrote to ewma_1_signed_error_time_series


In [140]:
model_to_graph = ['ewma_03_lw', 'ewma_03', 'ewma_01_lw', 'ewma_01', 'ewma_1']
err_df = pd.DataFrame(model_errors, columns=['model', 'end_date', 'battleground', 'mean_signed_error'])
err_df = err_df[np.isin(err_df['model'], model_to_graph)]
err_df = err_df[err_df['end_date'] == '2020-10-31'].reset_index(drop=True)[['model', 'battleground', 'mean_signed_error']]

In [145]:
fig = px.bar(err_df, x="model", y="mean_signed_error", color="battleground",
              labels={'model': 'Model', 'mean_signed_error': 'Mean Signed Error'},
             title=f'Mean Signed Error\n(Negative Means Democrat Optimistic)')
# fig.show()
filename = f'{"_".join(model_to_graph)}_mean_signed_error'
fig.write_image('figures/' + filename + '.png')
model_df.to_csv('figures/data/' + filename + '.csv', index=False)
open(f'figures/tables/{filename}.tex', 'w+').write(model_df.to_latex(index=False, caption=f'Mean Signed Error\n(Negative Means Democrat Optimistic)'))
print(f'Wrote to {filename}')

Wrote to ewma_03_lw_ewma_03_ewma_01_lw_ewma_01_ewma_1_mean_signed_error
